# Privacy-Preserving Record Linkage (PPRL) Demonstration

This notebook is the recommended walkthrough for the `pprl-superhero-example` demo. It shows how two organizations can link records **without exchanging raw identifiers** (like names or Social Security Numbers).

**Scenario:** Super Hero Hospital and Super Hero Pharmacy want to link patient records for care coordination while protecting privacy.

**Who does what:** In this demo, assume the **hospital** runs the overlap analysis step (it receives the pharmacy token file and compares tokens to find matches).

**Match policy:** OpenToken provides standard token rules (T1–T5), but it does **not** define a single, universal match policy. A match policy is what the parties agree on: **which token IDs must match (and how many)** before treating two records as the same person. This notebook shows strict matching (T1–T5) and an example of relaxing the policy.

**Custom tokens (optional):** Parties can define additional token rules beyond T1–T5, as long as both sides use the exact same token definitions and normalization rules (otherwise tokens won’t be comparable).

**Prefer a one-command run?** From this directory you can run `./run_end_to_end.sh` to generate data, tokenize, and analyze overlap end-to-end (see `README.md`).

> **Important demo disclaimer:** This example uses fully synthetic data and example secrets that are safe for illustration only. Do not reuse these keys or patterns in production. Real deployments must use strong key management, strict access controls around decryption and matching, and clear governance over who can run linkage jobs and how results are used.

In [ ]:
# Setup: imports + paths + optional PySpark bridge
import glob
import json
import os
import shutil
import subprocess
import sys
from pathlib import Path

import pandas as pd


def _find_demo_dir() -> Path:
    """Find the demo directory regardless of the current working directory."""
    start = Path.cwd().resolve()
    candidates = [start, *start.parents]
    for base in candidates:
        # Case 1: notebook opened from the demo directory
        if (base / "scripts" / "generate_superhero_datasets.py").exists():
            return base
        # Case 2: notebook opened from repo root (or elsewhere)
        demo = base / "demos" / "pprl-superhero-example"
        if (demo / "scripts" / "generate_superhero_datasets.py").exists():
            return demo
    raise FileNotFoundError("Could not locate demos/pprl-superhero-example")


demo_dir = _find_demo_dir()
scripts_dir = demo_dir / "scripts"
datasets_dir = demo_dir / "datasets"
outputs_dir = demo_dir / "outputs"

datasets_dir.mkdir(parents=True, exist_ok=True)
outputs_dir.mkdir(parents=True, exist_ok=True)

print(f"Demo directory: {demo_dir}")

# Demo-only secrets used by the in-notebook PySpark fast path.
# The CLI script path uses the secrets embedded in the demo scripts instead.
HASHING_SECRET = "HashingKey"
ENCRYPTION_KEY = "Secret-Encryption-Key-Goes-Here."  # Must be exactly 32 characters

pyspark_available = False
spark = None
OpenTokenProcessor = None
OpenTokenOverlapAnalyzer = None

try:
    from pyspark.sql import SparkSession  # type: ignore
    from opentoken_pyspark import OpenTokenProcessor  # type: ignore
    from opentoken_pyspark.overlap_analyzer import OpenTokenOverlapAnalyzer  # type: ignore

    pyspark_available = True
    spark = (
        SparkSession.builder
        .appName("PPRL-Superhero-Demo")
        .master("local[*]")
        .config("spark.sql.shuffle.partitions", "4")
        .config("spark.driver.memory", "2g")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    print("✓ PySpark + OpenToken PySpark Bridge available (optional fast path)")
except Exception as e:
    print(f"PySpark bridge not available (will use demo scripts): {type(e).__name__}")

## 2. Generate Superhero Datasets

Create two datasets (hospital and pharmacy) with a 40% overlap. The overlap represents patients that appear in both datasets.

In [ ]:
# Run the data generation script
result = subprocess.run(
    [sys.executable, str(scripts_dir / "generate_superhero_datasets.py")],
    cwd=str(demo_dir),
    capture_output=True,
    text=True,
    check=False,
 )

print(result.stdout)
if result.returncode != 0:
    print(f"Error: {result.stderr}")
    raise RuntimeError("Dataset generation failed")
print("✓ Datasets generated successfully!")

### Inspect the Generated Datasets

In [ ]:
# Load and display hospital dataset
hospital_df = pd.read_csv(datasets_dir / 'hospital_superhero_data.csv')
print(f"Hospital Dataset: {len(hospital_df)} records")
print(hospital_df.head())
print()

# Load and display pharmacy dataset
pharmacy_df = pd.read_csv(datasets_dir / 'pharmacy_superhero_data.csv')
print(f"Pharmacy Dataset: {len(pharmacy_df)} records")
print(pharmacy_df.head())

## 3. Tokenize the Datasets (in this notebook)

Each organization tokenizes their data independently. In this notebook, we tokenize using the **OpenToken PySpark Bridge**, which applies:
1. Normalization
2. HMAC-SHA256 hashing for a deterministic fingerprint layer
3. AES-256-GCM encryption (random IVs) for safe sharing

**Important:** Both organizations must use the same hashing secret and encryption key (shared securely ahead of time), otherwise tokens won’t be comparable.

**Note:** The end-to-end script (`run_end_to_end.sh`) runs a similar flow using the CLI scripts in this demo directory.

In [ ]:
print("Tokenizing datasets...")
print()

if pyspark_available and spark:
    # Tokenize datasets using OpenToken PySpark Bridge
    print("Using OpenToken PySpark Bridge...")
    print()

    # Configuration (demo secrets only)
    hashing_secret = HASHING_SECRET
    encryption_key = ENCRYPTION_KEY

    # Verify encryption key length
    if len(encryption_key) != 32:
        raise ValueError(f"Encryption key must be exactly 32 characters, got {len(encryption_key)}")

    # Initialize the OpenToken processor
    processor = OpenTokenProcessor(
        hashing_secret=hashing_secret,
        encryption_key=encryption_key,
    )
    print("✓ OpenToken Processor initialized")

    # Load hospital dataset into Spark
    print("Loading hospital dataset...")
    hospital_spark_df = spark.read.csv(
        str(datasets_dir / "hospital_superhero_data.csv"),
        header=True,
        inferSchema=True,
    )
    print(f"  Loaded {hospital_spark_df.count()} records")

    # Generate tokens for hospital dataset
    print("Generating hospital tokens...")
    hospital_tokens_spark = processor.process_dataframe(hospital_spark_df)
    hospital_token_count = hospital_tokens_spark.count()
    print(f"  Generated {hospital_token_count} tokens")

    # Save hospital tokens to CSV
    hospital_tokens_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(
        str(outputs_dir / "hospital_tokens_temp")
    )

    # Move the CSV file to the expected location
    csv_file = glob.glob(str(outputs_dir / "hospital_tokens_temp/*.csv"))[0]
    os.rename(csv_file, str(outputs_dir / "hospital_tokens.csv"))
    shutil.rmtree(str(outputs_dir / "hospital_tokens_temp"))
    print("✓ Hospital dataset tokenized")

    # Load pharmacy dataset into Spark
    print("Loading pharmacy dataset...")
    pharmacy_spark_df = spark.read.csv(
        str(datasets_dir / "pharmacy_superhero_data.csv"),
        header=True,
        inferSchema=True,
    )
    print(f"  Loaded {pharmacy_spark_df.count()} records")

    # Generate tokens for pharmacy dataset
    print("Generating pharmacy tokens...")
    pharmacy_tokens_spark = processor.process_dataframe(pharmacy_spark_df)
    pharmacy_token_count = pharmacy_tokens_spark.count()
    print(f"  Generated {pharmacy_token_count} tokens")

    # Save pharmacy tokens to CSV
    pharmacy_tokens_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(
        str(outputs_dir / "pharmacy_tokens_temp")
    )
    csv_file = glob.glob(str(outputs_dir / "pharmacy_tokens_temp/*.csv"))[0]
    os.rename(csv_file, str(outputs_dir / "pharmacy_tokens.csv"))
    shutil.rmtree(str(outputs_dir / "pharmacy_tokens_temp"))
    print("✓ Pharmacy dataset tokenized")
    print()
    print("✓ Tokenization completed successfully using PySpark!")
else:
    # Fall back to the standard demo scripts (Java CLI under the hood)
    print("Using standard demo scripts:")
    print("  - scripts/tokenize_hospital.sh")
    print("  - scripts/tokenize_pharmacy.sh")
    print()

    hospital_script = scripts_dir / "tokenize_hospital.sh"
    pharmacy_script = scripts_dir / "tokenize_pharmacy.sh"

    subprocess.run(["chmod", "+x", str(hospital_script)], cwd=str(demo_dir), check=False)
    subprocess.run(["chmod", "+x", str(pharmacy_script)], cwd=str(demo_dir), check=False)

    result = subprocess.run(
        [str(hospital_script)],
        cwd=str(demo_dir),
        capture_output=True,
        text=True,
        check=False,
    )
    print(result.stdout)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        raise RuntimeError("Hospital tokenization failed")

    result = subprocess.run(
        [str(pharmacy_script)],
        cwd=str(demo_dir),
        capture_output=True,
        text=True,
        check=False,
    )
    print(result.stdout)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        raise RuntimeError("Pharmacy tokenization failed")

    print("✓ Tokenization completed successfully using the demo scripts")

### Inspect Tokenized Data

In [ ]:
# Load tokenized hospital data
hospital_tokens = pd.read_csv(outputs_dir / 'hospital_tokens.csv')
print(f"Hospital Tokens: {len(hospital_tokens)} token rows (5 per patient)")
print(hospital_tokens.head(10))
print()

# Load tokenized pharmacy data
pharmacy_tokens = pd.read_csv(outputs_dir / 'pharmacy_tokens.csv')
print(f"Pharmacy Tokens: {len(pharmacy_tokens)} token rows (5 per patient)")
print(pharmacy_tokens.head(10))

## 4. Decrypt Tokens and Perform Overlap Analysis

To compare tokens across independently tokenized datasets:
1. **Decrypt** the encrypted tokens to reveal the underlying HMAC-SHA256 fingerprints
2. **Compare** those fingerprints to find matching records

**Why decryption is needed:** OpenToken uses random IVs for encryption, so even identical patients produce different encrypted token strings. Decryption reveals the deterministic fingerprint layer that can be compared for equality.

**Who runs this step (in this demo):** assume the **hospital** runs the overlap analysis step in a trusted environment (it needs access to the decryption key).

**Match policy reminder:** the set of token IDs you require to match (e.g., T1–T5, or a smaller subset) is a policy choice the parties agree on up front.

In [ ]:
print("Performing overlap analysis...")
print()

# Use the PySpark bridge if available; otherwise use the standard demo script.
if pyspark_available and spark:
    print("Using OpenToken PySpark Bridge...")
    print()

    try:
        # Load tokenized data into Spark DataFrames
        hospital_tokens_spark = spark.read.csv(
            str(outputs_dir / 'hospital_tokens.csv'),
            header=True,
            inferSchema=True,
        )
        pharmacy_tokens_spark = spark.read.csv(
            str(outputs_dir / 'pharmacy_tokens.csv'),
            header=True,
            inferSchema=True,
        )

        print(f"Hospital tokens loaded: {hospital_tokens_spark.count()} rows")
        print(f"Pharmacy tokens loaded: {pharmacy_tokens_spark.count()} rows")
        print()

        # Initialize the overlap analyzer with the encryption key
        encryption_key = "Secret-Encryption-Key-Goes-Here."  # Same key used for tokenization
        analyzer = OpenTokenOverlapAnalyzer(encryption_key)

        print("Decrypting and matching tokens...")

        # matching_rules specifies which token IDs must ALL match
        results = analyzer.analyze_overlap(
            hospital_tokens_spark,
            pharmacy_tokens_spark,
            matching_rules=["T1", "T2", "T3", "T4", "T5"],
            dataset1_name="Hospital",
            dataset2_name="Pharmacy",
        )

        print("✓ Overlap analysis completed using OpenToken PySpark Bridge!")
        print()

        # Display results
        print("=" * 70)
        print("OVERLAP ANALYSIS RESULTS")
        print("=" * 70)
        print(f"Total Hospital records: {results['total_records_dataset1']}")
        print(f"Total Pharmacy records: {results['total_records_dataset2']}")
        print(f"Matching Hospital records: {results['matching_records_dataset1']}")
        print(f"Matching Pharmacy records: {results['matching_records_dataset2']}")
        print(f"Overlap percentage: {results['overlap_percentage']:.1f}%")
        print(f"Records unique to Hospital: {results['unique_to_dataset1']}")
        print(f"Records unique to Pharmacy: {results['unique_to_dataset2']}")
        print("=" * 70)
        print()

        # Get the matches DataFrame
        matches_df_spark = results['matches']
        print("Sample matches (first 10):")
        matches_df_spark.show(10, truncate=False)
        print()

        # Save results to CSV
        matches_df_spark.coalesce(1).write.mode("overwrite").option("header", "true").csv(
            str(outputs_dir / 'matching_records_temp')
        )
        import glob
        import shutil

        csv_file = glob.glob(str(outputs_dir / 'matching_records_temp/*.csv'))[0]
        shutil.copy(csv_file, str(outputs_dir / 'matching_records.csv'))
        shutil.rmtree(str(outputs_dir / 'matching_records_temp'))
        print("✓ Results saved to outputs/matching_records.csv")

    except Exception as e:
        import traceback
        print(f"Error during PySpark analysis: {e}")
        print(traceback.format_exc())
        print()
        print("Falling back to the standard demo overlap script...")
        pyspark_available = False  # force fallback below


if not pyspark_available:
    print("Using standard demo script: scripts/analyze_overlap.py")
    analyze_script = scripts_dir / 'analyze_overlap.py'
    result = subprocess.run(
        ['python', str(analyze_script)],
        cwd=str(demo_dir),
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        raise RuntimeError("Overlap analysis failed")
    print("✓ Overlap analysis completed successfully using the demo script")

### View Matching Results

In [ ]:
# Load and display matching results
matches_df = pd.read_csv(outputs_dir / 'matching_records.csv')

print(f"Total Matching Pairs: {len(matches_df)}")
print()
print("First 10 matching records:")
print(matches_df.head(10))
print()

# Summary statistics
hospital_count = len(hospital_df)
pharmacy_count = len(pharmacy_df)
unique_hospital_matches = matches_df['Hospital_RecordId'].nunique()
unique_pharmacy_matches = matches_df['Pharmacy_RecordId'].nunique()

print("Summary Statistics:")
print(f"- Hospital records with matches: {unique_hospital_matches} out of {hospital_count}")
print(f"- Pharmacy records with matches: {unique_pharmacy_matches} out of {pharmacy_count}")
print(f"- Overlap percentage (hospital): {(unique_hospital_matches / hospital_count * 100):.1f}%")
print(f"- Overlap percentage (pharmacy): {(unique_pharmacy_matches / pharmacy_count * 100):.1f}%")

## 5. Alternative Analysis: Relaxed Matching Rules

Let's compare the results when we relax the matching requirements. Instead of requiring all 5 tokens (T1-T5) to match, we'll only require T1, T2, T3, and T5 to match (excluding T4).

**Use case**: This can be useful when one attribute (like postal code in T4) might have data quality issues or when you want to cast a wider net for potential matches.

In [ ]:
# Alternative overlap analysis with relaxed matching rules (requires PySpark bridge).
print("Performing alternative overlap analysis (T1, T2, T3, T5 only)...")
print()

if not (pyspark_available and spark):
    print("Skipping: PySpark bridge not available in this environment.")
elif "results" not in globals():
    print("Skipping: run the main overlap analysis cell first.")
else:
    try:
        # Load tokenized data into Spark DataFrames
        hospital_tokens_spark_alt = spark.read.csv(
            str(outputs_dir / "hospital_tokens.csv"),
            header=True,
            inferSchema=True,
        )
        pharmacy_tokens_spark_alt = spark.read.csv(
            str(outputs_dir / "pharmacy_tokens.csv"),
            header=True,
            inferSchema=True,
        )

        # Initialize the overlap analyzer
        analyzer_alt = OpenTokenOverlapAnalyzer(ENCRYPTION_KEY)

        print("Decrypting and matching tokens (excluding T4)...")

        # Analyze overlap with relaxed rules - only T1, T2, T3, T5
        results_alt = analyzer_alt.analyze_overlap(
            hospital_tokens_spark_alt,
            pharmacy_tokens_spark_alt,
            matching_rules=["T1", "T2", "T3", "T5"],  # Excluding T4
            dataset1_name="Hospital",
            dataset2_name="Pharmacy",
        )

        print("✓ Alternative overlap analysis completed!")
        print()

        # Compare with original results
        print("=" * 70)
        print("COMPARISON: All 5 Tokens vs. 4 Tokens (excluding T4)")
        print("=" * 70)
        print(f"Strict matching (T1-T5): {results['matching_records_dataset1']} hospital matches")
        print(f"Relaxed matching (T1,T2,T3,T5): {results_alt['matching_records_dataset1']} hospital matches")
        print(f"Additional matches found: {results_alt['matching_records_dataset1'] - results['matching_records_dataset1']}")
        print("=" * 70)
        print()

        # Save alternative results
        matches_df_spark_alt = results_alt["matches"]
        matches_df_spark_alt.coalesce(1).write.mode("overwrite").option("header", "true").csv(
            str(outputs_dir / "matching_records_alt_temp")
        )
        csv_file_alt = glob.glob(str(outputs_dir / "matching_records_alt_temp/*.csv"))[0]
        shutil.copy(csv_file_alt, str(outputs_dir / "matching_records_alt.csv"))
        shutil.rmtree(str(outputs_dir / "matching_records_alt_temp"))
        print("✓ Alternative results saved to outputs/matching_records_alt.csv")
    except Exception as e:
        import traceback
        print(f"Error during alternative analysis: {e}")
        print(traceback.format_exc())

### Interpreting the Alternative Analysis

The alternative analysis uses only 4 tokens (T1, T2, T3, T5) instead of all 5:
- **T1**: FirstName + LastName + Sex + BirthDate
- **T2**: FirstName + LastName + PostalCode
- **T3**: FirstName + LastName + SocialSecurityNumber
- **T4**: ❌ *EXCLUDED* - BirthDate + Sex + PostalCode
- **T5**: BirthDate + Sex + SocialSecurityNumber

By excluding T4, we're being more lenient about postal code consistency. This might find additional matches where:
- Postal codes have typos or formatting differences
- People have moved between visits
- Data entry errors occurred

However, this also increases the risk of false positives.

## 6. Understand the Results

Let's look at what a match actually means by examining some matched records in detail.

In [ ]:
# Get a sample matched record
if len(matches_df) > 0:
    sample_match = matches_df.iloc[0]
    hospital_record_id = sample_match['Hospital_RecordId']
    pharmacy_record_id = sample_match['Pharmacy_RecordId']
    
    # Get the original records
    hospital_match = hospital_df[hospital_df['RecordId'] == hospital_record_id]
    pharmacy_match = pharmacy_df[pharmacy_df['RecordId'] == pharmacy_record_id]
    
    if len(hospital_match) == 0 or len(pharmacy_match) == 0:
        print(f"Warning: Could not find matching records in original datasets")
        print(f"Hospital RecordId {hospital_record_id} found: {len(hospital_match) > 0}")
        print(f"Pharmacy RecordId {pharmacy_record_id} found: {len(pharmacy_match) > 0}")
    else:
        hospital_record = hospital_match.iloc[0]
        pharmacy_record = pharmacy_match.iloc[0]
        
        print("Sample Match:")
        print(f"Hospital Record ID: {hospital_record_id}")
        print(f"Hospital Patient: {hospital_record['FirstName']} {hospital_record['LastName']}")
        print(f"DOB: {hospital_record['BirthDate']}, SSN: {hospital_record['SocialSecurityNumber']}")
        print()
        print(f"Pharmacy Record ID: {pharmacy_record_id}")
        print(f"Pharmacy Patient: {pharmacy_record['FirstName']} {pharmacy_record['LastName']}")
        print(f"DOB: {pharmacy_record['BirthDate']}, SSN: {pharmacy_record['SocialSecurityNumber']}")
        print()
        print("✓ All 5 tokens matched, confirming this is the same patient!")
else:
    print("No matches found. This could happen if:")
    print("- Different hashing/encryption keys were used")
    print("- Data validation rejected records with invalid attributes")

## 7. Advanced PySpark Transformations (Optional)

If PySpark is available, we can perform distributed transformations on the tokenized data for large-scale analysis.


In [ ]:
# PySpark-based transformations for distributed processing
if pyspark_available and spark:
    try:
        from pyspark.sql.functions import col, count as spark_count
        
        print("Performing distributed token analysis with PySpark...")
        print()
        
        # Load the tokenized data (all columns as strings to avoid type inference issues)
        hospital_tokens_spark = spark.read.csv(
            str(outputs_dir / 'hospital_tokens.csv'),
            header=True,
            inferSchema=False  # Keep all as strings
        )
        pharmacy_tokens_spark = spark.read.csv(
            str(outputs_dir / 'pharmacy_tokens.csv'),
            header=True,
            inferSchema=False  # Keep all as strings
        )
        
        # Analyze token distribution in hospital dataset
        print("Hospital Token Distribution:")
        hospital_tokens_spark.groupBy("RuleId").agg(spark_count("*").alias("count")).orderBy("RuleId").show()
        print()
        
        # Analyze token distribution in pharmacy dataset
        print("Pharmacy Token Distribution:")
        pharmacy_tokens_spark.groupBy("RuleId").agg(spark_count("*").alias("count")).orderBy("RuleId").show()
        print()
        
        # Count unique records
        hospital_unique = hospital_tokens_spark.select("RecordId").distinct().count()
        pharmacy_unique = pharmacy_tokens_spark.select("RecordId").distinct().count()
        print(f"Unique records - Hospital: {hospital_unique}, Pharmacy: {pharmacy_unique}")
        
    except Exception as e:
        print(f"Note: Advanced transformations not available - {type(e).__name__}")
        print("This is optional and does not affect the core PPRL workflow.")
else:
    print("PySpark not available for advanced transformations.")
    print("Core PPRL analysis completed successfully using pandas.")

## 8. Privacy and Security Summary

This demonstration shows how OpenToken enables privacy-preserving record linkage:

### What was protected:
- ✓ Raw patient data (names, SSNs, birthdates) was never shared between organizations
- ✓ HMAC-SHA256 hashes cannot be reversed to recover original data
- ✓ Encryption key controls who can decrypt and perform linkage

### What was shared:
- • Encrypted tokens for secure transmission
- • Matching statistics showing overlap counts
- • Metadata with summary information (not raw data)

### Key security principles:
1. **Strong Encryption**: AES-256-GCM with random IVs prevents pattern analysis
2. **Key Management**: Secure sharing and storage of encryption/hashing keys
3. **Deterministic Hashing**: HMAC-SHA256 enables comparison without raw data
4. **Access Control**: Only authorized parties can decrypt tokens

### PySpark Bridge Benefits:
- **Distributed Processing**: Handle large datasets across multiple nodes
- **Parallel Decryption**: Efficiently decrypt millions of tokens
- **Scalable Analysis**: Perform overlap analysis on enterprise-scale data
- **Integration**: Native Spark SQL for additional transformations

## 9. Customization Examples

You can customize this demo by modifying the scripts:

### Change dataset size and overlap:
Edit `scripts/generate_superhero_datasets.py`:
```python
num_hospital = 500  # Different size
num_pharmacy = 600
overlap_percentage = 0.50  # 50% overlap instead of 40%
```

### Use different encryption keys:
Edit `scripts/tokenize_datasets.sh`:
```bash
HASH_KEY="YourCustomHashingKey"
ENCRYPTION_KEY="YourCustomEncryptionKey-32"
```

**Important**: Both organizations must use the same keys for tokens to match!

### Scale with PySpark:
For large datasets, ensure PySpark is installed:
```bash
pip install pyspark opentoken-pyspark
```

The notebook will automatically use distributed processing if available.

## 10. Next Steps

This PPRL demo can be adapted for:
- Healthcare: Hospital-to-hospital patient matching
- Insurance: Claims linkage across providers
- Research: Multi-site study participant matching
- Government: Cross-agency identity resolution
- Financial Services: Anti-fraud systems

### With PySpark Bridge:
- Scale to petabyte-level datasets
- Distribute tokenization across clusters
- Parallel overlap analysis
- Real-time record linkage pipelines

For more information, see the [README.md](./README.md) in this directory and the [main OpenToken documentation](../../README.md).

<!-- notebook-edit-test -->